## Setup

In [2]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import pandas as pd
import torch
import torch.nn as nn
import xlrd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Import Data

In [ ]:
df = pd.read_excel ('default of credit card clients.xls',header = 1)

## Data Processing

### check missing values

In [ ]:
df.shape[0] - df.dropna().shape[0]

- No missing values

### drop some redundant attributes

In [ ]:
df.columns

In [ ]:
df.drop(columns = ['ID'],inplace = True)

In [ ]:
df.describe()

- History of past payment arrtibutes with negative record should be set 0, as it also means 'pay duly' according to the data description.
- Convert `SEX`, `EDUCATION` and `MARRIAGE` columns to category data, and delete invalid data.

In [ ]:
pay_cols = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']
pay_cols = [col.upper() for col in pay_cols]


for col in pay_cols:
    fil = (df[col] < 0)
    df.loc[fil, col] = 0

In [ ]:
drop_id_1 = (df['EDUCATION'] != 1 ) & (df['EDUCATION'] != 2 ) & (df['EDUCATION'] != 3) & (df['EDUCATION'] != 4 )
drop_id_2 = (df['MARRIAGE'] != 1 ) & (df['MARRIAGE'] != 2 ) & (df['MARRIAGE'] != 3)
keep_id = -(drop_id_1 | drop_id_2)
df = df[keep_id]

In [ ]:
df.loc[df['SEX'] == 1,'SEX'] = 'male'
df.loc[df['SEX'] == 2,'SEX'] = 'female'
df.loc[df['EDUCATION'] == 1,'EDUCATION'] = 'graduate'
df.loc[df['EDUCATION'] == 2,'EDUCATION'] = 'university'
df.loc[df['EDUCATION'] == 3,'EDUCATION'] = 'high'
df.loc[df['EDUCATION'] == 4,'EDUCATION'] = 'others'
df.loc[df['MARRIAGE'] == 1,'MARRIAGE'] = 'married'
df.loc[df['MARRIAGE'] == 2,'MARRIAGE'] = 'single'
df.loc[df['MARRIAGE'] == 3,'MARRIAGE'] = 'others'

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df = df.astype(float)
df = df.dropna()
scale = MinMaxScaler()
df_to_scale = df
scaled = scale.fit_transform(df_to_scale)
scaled_df = pd.DataFrame(scaled, columns=df_to_scale.columns)

## Split data

In [ ]:
test_size = round(0.2 * len(scaled_df))
train, test = train_test_split(scaled_df, test_size=test_size, random_state=0, shuffle=True)

label = 'default payment next month'

x_train, y_train = train.drop(label, axis=1), train[label]
x_test, y_test = test.drop(label, axis=1), test[label]

In [ ]:
scaled_df.columns

In [ ]:
x_train.shape,x_test.shape

## Build Models

In [1]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

NameError: name 'os' is not defined

In [ ]:
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(x_train.values), 
                       torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(x_test.values))

In [20]:
BATCH_SIZE = 64
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

NameError: name 'train_data' is not defined